In [87]:
import sys
import os
sys.path.append('/home/jijoo/miniconda3/envs/theor/lib/python3.8/site-packages/cana') # cana package 위치
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

import math
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2g}'.format
import matplotlib as mpl
import matplotlib.pyplot as plt
import random
import graphviz
from IPython.display import HTML
#
import cana
from cana.datasets.bio import *
from cana.drawing.canalizing_map import draw_canalizing_map_graphviz
import networkx as nx
import par_helper as ph
import jReversion as jR
from LDOI import BooleanDOI_processing as BDOIp
from LDOI import BooleanDOI_DOI as BDOId

In [111]:
def has_only_composite_predecessors(G, node):
    predecessors = G.predecessors(node)
    non_composite = [x for x in predecessors if x.find('_') < 0]
    return len(non_composite) == 0

def unstable_candidate(G, node):
    ON_candidate = has_only_composite_predecessors(G, node)
    OFF_candidate = has_only_composite_predecessors(G, '~'+node)
    
    if ON_candidate and OFF_candidate:
        return [node, '~'+node]
    elif ON_candidate:
        return node
    elif OFF_candidate:
        return '~'+node
    
    return []

def monostable_motif(G, node):
    candidate = unstable_candidate(G, node)
    if type(candidate) == list:
        return
        
    composite_nodes = set([x for x in G.predecessors(candidate) if x.find('_') >= 0])
    monostable_motif = set()
#     for cycle in nx.simple_cycles(G):
#         if cycle.count(candidate) > 0 :
#             if len(composite_nodes.intersection(set(cycle))) > 0:
#                 monostable_motif = monostable_motif.union(cycle)
    
    for composite in composite_nodes:
        if nx.has_path(G, source=BDOId.Negation_in_expanded(candidate), target=composite):
            for negative_feedback in nx.all_shortest_paths(G, source=BDOId.Negation_in_expanded(candidate), target=composite):
                monostable_motif = monostable_motif.union(negative_feedback)
        if nx.has_path(G, source=candidate, target=composite):
            for positive_feedback in nx.all_shortest_paths(G, source=candidate, target=composite):
                monostable_motif = monostable_motif.union(positive_feedback)
                
    return monostable_motif

def monostable_motif2(G, node):
    candidate = node
#     if type(candidate) == list:
#         return
        
    composite_nodes = set([x for x in G.predecessors(candidate)])
    monostable_motif = set()
#     for cycle in nx.simple_cycles(G):
#         if cycle.count(candidate) > 0 :
#             if len(composite_nodes.intersection(set(cycle))) > 0:
#                 monostable_motif = monostable_motif.union(cycle)
    
    for composite in composite_nodes:
        if nx.has_path(G, source=BDOId.Negation_in_expanded(candidate), target=composite):
            for negative_feedback in nx.all_shortest_paths(G, source=BDOId.Negation_in_expanded(candidate), target=composite):
                monostable_motif = monostable_motif.union(negative_feedback)
        if nx.has_path(G, source=candidate, target=composite):
            for positive_feedback in nx.all_shortest_paths(G, source=candidate, target=composite):
                monostable_motif = monostable_motif.union(positive_feedback)
    
    candidate = '~' + node
#     if type(candidate) == list:
#         return
        
    composite_nodes = set([x for x in G.predecessors(candidate)])
#     monostable_motif = set()
#     for cycle in nx.simple_cycles(G):
#         if cycle.count(candidate) > 0 :
#             if len(composite_nodes.intersection(set(cycle))) > 0:
#                 monostable_motif = monostable_motif.union(cycle)
    
    for composite in composite_nodes:
        if nx.has_path(G, source=BDOId.Negation_in_expanded(candidate), target=composite):
            for negative_feedback in nx.all_shortest_paths(G, source=BDOId.Negation_in_expanded(candidate), target=composite):
                monostable_motif = monostable_motif.union(negative_feedback)
        if nx.has_path(G, source=candidate, target=composite):
            for positive_feedback in nx.all_shortest_paths(G, source=candidate, target=composite):
                monostable_motif = monostable_motif.union(positive_feedback)
                
    return monostable_motif

In [57]:
# len(scc_ex)

46

In [70]:
# # print(monostable_motif(scc_ex, mapping['n07']))
# monostable_network = nx.subgraph(scc_ex, monostable_motif(scc_ex, mapping[interest_node])).copy()
# nx.write_gml(monostable_network, '../networks/' + network_name + '_' + interest_node + '_monostable.gml')

In [112]:
network_name = 'monoFVS1'
interest_node = 'n06'

model_name = network_name + '.txt'
prefix, suffix = 'n', 'n'

# num_inputs = 0
# input_nodes = []
lines = []

# lines.append('# Internal nodes\n')
with open('../models/' + model_name, 'r') as exf:
    for line in exf:
        line = ' '.join(line.split())
#         logic = line.replace(' =', '*=')

        lines.append(line)


Gread, read_nodes = BDOIp.form_network(lines, sorted_nodename=False)

# calculate the mapping from string nodename to index
mapping = {}  # nodename to number index
inverse_mapping = {}  # number index to nodename
read_nodes_dict = {}
inverse_read_nodes = {}
for i, node in enumerate(read_nodes):
    index = prefix + str(i) + suffix
    mapping[node] = index
    inverse_mapping[index] = node
    mapping['~' + node] = '~' + index
    inverse_mapping['~' + index] = '~' + node
    read_nodes_dict[i] = node
    inverse_read_nodes[node] = i


interest_cycles = [cycle for cycle in nx.simple_cycles(G=Gread) if cycle.count(inverse_read_nodes[interest_node]) > 0]

interest_scc = set()
for cycle in interest_cycles:
    temp = set(cycle)
    interest_scc.update(temp)

print(interest_scc)

scc_ex = BDOIp.Get_expanded_subnetwork(Gread=Gread, nodes=interest_scc)
monostable_network = nx.subgraph(scc_ex, monostable_motif2(scc_ex, mapping[interest_node])).copy()

isComposite = dict()
for node in monostable_network.nodes():
    isComposite[node] = node.find('_')>0
nx.set_node_attributes(monostable_network, isComposite, name='is_composite')
monostable_network = nx.relabel_nodes(monostable_network, inverse_mapping)
nx.write_gml(monostable_network, '../networks/' + network_name + '_' + interest_node + '_monostable.gml')

{0, 1, 2, 3, 4, 5, 6, 7, 8}


In [102]:
inverse_mapping[unstable_candidate(scc_ex, mapping[interest_node])]

'n06'

In [104]:
GExpanded = BDOIp.Get_expanded_network(Gread)
isComposite = dict()
for node in GExpanded.nodes():
    isComposite[node] = node.find('_')>0
nx.set_node_attributes(GExpanded, isComposite, name='is_composite')
GExpanded = nx.relabel_nodes(GExpanded, inverse_mapping)
nx.write_gml(GExpanded, '../networks/' + network_name + '_expanded.gml')